# Neighbourhoods in Toronto

In [19]:
!pip install beautifulsoup4
# !pip install lxml
#!pip install html5lib
# !pip install requests
! pip install geocoder
# #try to solve [Errno 2] No such file or directory: 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'#
#!pip install html.parser 

     |████████████████████████████████| 98 kB 3.0 MB/s eta 0:00:011


In [8]:
import numpy as np 

import pandas as pd
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

import json

#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

#from bs2json import bs2json
#import wikipedia as wp
from bs4 import BeautifulSoup

print('Libraries imported.')

Libraries imported.


In [9]:
#url = 'https://en.wikipedia.org/wiki/New_York_City'
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

html_text =requests. get(url).text
soup = BeautifulSoup(html_text, 'html.parser')

print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of postal codes of Canada: M - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"f86bbd4c-afe7-4682-99c1-4e8c73f419bf","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":1013111980,"wgRevisionId":1013111980,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles with short description","Short description is different from Wikidata","Wikipedia

In [39]:
table_contents=[]
table=soup.find('table')
#table

for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

#print(table_contents)

df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})
df

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto Business,Enclave of M4L
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [51]:
from itertools import chain

# return list from series of comma-separated strings
def chainer(s):
    return list(chain.from_iterable(s.str.split(',')))

# calculate lengths of splits
lens = df['Neighborhood'].str.split(',').map(len)

# create new dataframe, repeating or chaining as appropriate
res = pd.DataFrame({'PostalCode': np.repeat(df['PostalCode'], lens),
                    'Borough': np.repeat(df['Borough'], lens),
                    'Neighborhood': chainer(df['Neighborhood'])})

res=res.reset_index()
res

,index,PostalCode,Borough,Neighborhood
0,0,M3A,North York,Parkwoods
1,1,M4A,North York,Victoria Village
2,2,M5A,Downtown Toronto,Regent Park
3,2,M5A,Downtown Toronto,Harbourfront
4,3,M6A,North York,Lawrence Manor
...,...,...,...,...
211,102,M8Z,Etobicoke,Mimico NW
212,102,M8Z,Etobicoke,The Queensway West
213,102,M8Z,Etobicoke,South of Bloor
214,102,M8Z,Etobicoke,Kingsway Park South West


In [54]:
import geocoder # import geocoder
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

latitude=[]
longitude=[]
for code in res['Neighborhood']:
    g = geocoder.arcgis('{}, Toronto, Ontario'.format(code))
    print(code, g.latlng)
    while (g.latlng is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(code))
        print(code, g.latlng)
    latlng = g.latlng
    latitude.append(latlng[0])
    longitude.append(latlng[1])
    

Parkwoods [43.6865884896713, -79.40999620161057]
Victoria Village [43.73154000000005, -79.31427999999994]
Regent Park [43.660690000000045, -79.36030999999997]
 Harbourfront [43.639230000000055, -79.38306999999998]
Lawrence Manor [43.72294000000005, -79.43115999999998]
 Lawrence Heights [43.72357000000005, -79.43710999999996]
Ontario Provincial Government [43.72509000000008, -79.48208999999997]
Islington Avenue [43.738221166575215, -79.56573343932973]
Malvern [43.81023000000005, -79.22037999999998]
 Rouge [43.807660000000055, -79.17404999999997]
Don Mills North [43.705685127473515, -79.33385691603588]
Parkview Hill [43.70464000000004, -79.32266999999996]
 Woodbine Gardens [43.70626000000004, -79.30090999999999]
Garden District [43.65794000000005, -79.37561999999997]
 Ryerson [43.64884662065743, -79.40249206410026]
Glencairn [43.708432596237074, -79.44644871017785]
West Deane Park [43.65297000000004, -79.55741999999998]
 Princess Gardens [43.77174903486855, -79.3973453485166]
 Martin Gro

Milliken [43.82325000000003, -79.27728999999994]
 Agincourt North [43.809300000000064, -79.26706999999999]
 Steeles East [43.82071346589683, -79.31991803631195]
 L'Amoreaux East [43.79976754912674, -79.31004752175751]
Summerhill West [43.61185291023563, -79.48875424435107]
 Rathnelly [43.67873567692022, -79.40097019517378]
 South Hill [43.67652863013039, -79.37918906679771]
 Forest Hill SE [43.689910000000054, -79.42466999999994]
 Deer Park [43.67797000000007, -79.40596999999997]
CN Tower [43.642630000000054, -79.38714999999996]
 King and Spadina [43.645440043716434, -79.39513995283988]
 Railway Lands [43.648690000000045, -79.38543999999996]
 Harbourfront West [43.639230000000055, -79.38306999999998]
 Bathurst Quay [43.63546000000008, -79.39810999999997]
 South Niagara [43.6415411839556, -79.40374346684759]
 Island airport [43.632100144918965, -79.39585076724106]
New Toronto [43.60143000000005, -79.50924999999995]
 Mimico South [43.61299121979061, -79.49303166611847]
 Humber Bay Shores

In [55]:
res['Latitude'] = latitude
res['Longitude'] = longitude
res

,index,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,0,M3A,North York,Parkwoods,43.686588,-79.409996
1,1,M4A,North York,Victoria Village,43.731540,-79.314280
2,2,M5A,Downtown Toronto,Regent Park,43.660690,-79.360310
3,2,M5A,Downtown Toronto,Harbourfront,43.639230,-79.383070
4,3,M6A,North York,Lawrence Manor,43.722940,-79.431160
...,...,...,...,...,...,...
211,102,M8Z,Etobicoke,Mimico NW,43.617290,-79.498850
212,102,M8Z,Etobicoke,The Queensway West,43.609787,-79.567084
213,102,M8Z,Etobicoke,South of Bloor,43.649266,-79.502830
214,102,M8Z,Etobicoke,Kingsway Park South West,43.657385,-79.503383


In [56]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(res['Borough'].unique()),
        res.shape[0]
    )
)

The dataframe has 15 boroughs and 216 neighborhoods.


In [59]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="Toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [65]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=13)

# add markers to map
for lat, lng, borough, neighborhood in zip(res['Latitude'], res['Longitude'], res['Borough'], res['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [78]:
Downtown_Toronto_data = res[res['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
Downtown_Toronto_data.head()

,index,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,2,M5A,Downtown Toronto,Regent Park,43.660690,-79.360310
1,2,M5A,Downtown Toronto,Harbourfront,43.639230,-79.383070
2,9,M5B,Downtown Toronto,Garden District,43.657940,-79.375620
3,9,M5B,Downtown Toronto,Ryerson,43.648847,-79.402492
4,15,M5C,Downtown Toronto,St. James Town,43.671100,-79.373590


In [67]:
address = 'Downtown Toronto, Toronto'

geolocator = Nominatim(user_agent="Downtown_Toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Downtown Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Downtown Toronto are 43.6541737, -79.38081164513409.


In [70]:
# create map of Manhattan using latitude and longitude values
map_Downtown_Toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, label in zip(Downtown_Toronto_data['Latitude'], Downtown_Toronto_data['Longitude'], Downtown_Toronto_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Downtown_Toronto)  
    
map_Downtown_Toronto

In [79]:
CLIENT_ID = 'MJDQMV351W4LKAO3XY5M2E1KBTPHQ0YFR3GJTSBDBHKQDSTE' # your Foursquare ID
CLIENT_SECRET = '2021GFSKHLSSDNMZGT50A3UNMI2S1PY53SL1CAH0XKKEPQ1Y' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: MJDQMV351W4LKAO3XY5M2E1KBTPHQ0YFR3GJTSBDBHKQDSTE
CLIENT_SECRET:2021GFSKHLSSDNMZGT50A3UNMI2S1PY53SL1CAH0XKKEPQ1Y


In [80]:
Downtown_Toronto_data.loc[0, 'Neighborhood']

'Regent Park'

In [81]:
neighborhood_latitude = Downtown_Toronto_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = Downtown_Toronto_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = Downtown_Toronto_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Regent Park are 43.660690000000045, -79.36030999999997.


In [82]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius


url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=MJDQMV351W4LKAO3XY5M2E1KBTPHQ0YFR3GJTSBDBHKQDSTE&client_secret=2021GFSKHLSSDNMZGT50A3UNMI2S1PY53SL1CAH0XKKEPQ1Y&v=20180605&ll=43.660690000000045,-79.36030999999997&radius=500&limit=100'

In [83]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '606c0c0fe0f4006d378ec14c'},
 'response': {'headerLocation': 'Regent Park',
  'headerFullLocation': 'Regent Park, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 22,
  'suggestedBounds': {'ne': {'lat': 43.66519000450005,
    'lng': -79.35410133531909},
   'sw': {'lat': 43.65618999550004, 'lng': -79.36651866468085}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5092fe87e4b0476c6e7375e2',
       'name': 'Regent Park Aquatic Centre',
       'location': {'address': '640 Dundas St. East',
        'crossStreet': 'at Sumach St.',
        'lat': 43.6606003130796,
        'lng': -79.36139162915349,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.6606003130796,
          'lng': -79.36139162915349

In [85]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [86]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

<ipython-input-86-561c05f0fdd1>:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues = json_normalize(venues) # flatten JSON


,name,categories,lat,lng
0,Regent Park Aquatic Centre,Pool,43.660600,-79.361392
1,Sumach Espresso,Coffee Shop,43.658135,-79.359515
2,Daniels Spectrum,Performing Arts Venue,43.660137,-79.361808
3,Thai To Go,Thai Restaurant,43.663418,-79.360710
4,Sukhothai,Thai Restaurant,43.658444,-79.365681


In [87]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

22 venues were returned by Foursquare.


In [89]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [90]:
# type your answer here
Downtown_Toronto_venues = getNearbyVenues(names=Downtown_Toronto_data['Neighborhood'],
                                   latitudes=Downtown_Toronto_data['Latitude'],
                                   longitudes=Downtown_Toronto_data['Longitude']
                                  )

Regent Park
 Harbourfront
Garden District
 Ryerson
St. James Town
Berczy Park
Central Bay Street
Christie
Richmond
 Adelaide
 King
Harbourfront East
 Union Station
 Toronto Islands
Toronto Dominion Centre
 Design Exchange
Commerce Court
 Victoria Hotel
University of Toronto
 Harbord
Kensington Market
 Chinatown
 Grange Park
CN Tower
 King and Spadina
 Railway Lands
 Harbourfront West
 Bathurst Quay
 South Niagara
 Island airport
Rosedale
St. James Town
 Cabbagetown
First Canadian Place
 Underground city
Church and Wellesley


In [91]:
print(Downtown_Toronto_venues.shape)
Downtown_Toronto_venues.head()

(2524, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Regent Park,43.66069,-79.36031,Regent Park Aquatic Centre,43.660600,-79.361392,Pool
1,Regent Park,43.66069,-79.36031,Sumach Espresso,43.658135,-79.359515,Coffee Shop
2,Regent Park,43.66069,-79.36031,Daniels Spectrum,43.660137,-79.361808,Performing Arts Venue
3,Regent Park,43.66069,-79.36031,Thai To Go,43.663418,-79.360710,Thai Restaurant
4,Regent Park,43.66069,-79.36031,Sukhothai,43.658444,-79.365681,Thai Restaurant


In [92]:
print('There are {} uniques categories.'.format(len(Downtown_Toronto_venues['Venue Category'].unique())))

There are 253 uniques categories.


In [93]:
# one hot encoding
Downtown_Toronto_onehot = pd.get_dummies(Downtown_Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Downtown_Toronto_onehot['Neighborhood'] = Downtown_Toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Downtown_Toronto_onehot.columns[-1]] + list(Downtown_Toronto_onehot.columns[:-1])
Downtown_Toronto_onehot = Downtown_Toronto_onehot[fixed_columns]

Downtown_Toronto_onehot.head()

,Yoga Studio,Accessories Store,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Animal Shelter,Antique Shop,...,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [94]:
Downtown_Toronto_onehot.shape

(2524, 253)

In [122]:
Downtown_Toronto_grouped = Downtown_Toronto_onehot.groupby('Neighborhood').mean().reset_index()
Downtown_Toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Animal Shelter,...,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,Adelaide,0.030000,0.01,0.000000,0.0000,0.000000,0.000000,0.000,0.010000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.020000,0.010000,0.00
1,Bathurst Quay,0.000000,0.00,0.037037,0.0000,0.037037,0.037037,0.000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00
2,Cabbagetown,0.000000,0.00,0.000000,0.0000,0.000000,0.000000,0.000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00
3,Chinatown,0.041667,0.00,0.000000,0.0000,0.000000,0.000000,0.000,0.000000,0.000000,...,0.000000,0.031250,0.010417,0.000000,0.020833,0.00,0.010417,0.000000,0.000000,0.00
4,Design Exchange,0.000000,0.00,0.000000,0.0000,0.000000,0.000000,0.000,0.030000,0.000000,...,0.000000,0.020000,0.000000,0.000000,0.000000,0.00,0.010000,0.000000,0.000000,0.00
5,Grange Park,0.010753,0.00,0.000000,0.0000,0.000000,0.000000,0.000,0.000000,0.000000,...,0.010753,0.010753,0.010753,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00
6,Harbord,0.000000,0.00,0.000000,0.0000,0.000000,0.000000,0.000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00
7,Harbourfront,0.010000,0.00,0.000000,0.0000,0.000000,0.000000,0.000,0.000000,0.000000,...,0.000000,0.010000,0.000000,0.000000,0.000000,0.01,0.010000,0.000000,0.000000,0.00
8,Harbourfront West,0.010000,0.00,0.000000,0.0000,0.000000,0.000000,0.000,0.000000,0.000000,...,0.000000,0.010000,0.000000,0.000000,0.000000,0.01,0.010000,0.000000,0.000000,0.00
9,Island airport,0.000000,0.00,0.062500,0.0625,0.125000,0.125000,0.125,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00


In [123]:
num_top_venues = 5

for hood in Downtown_Toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Downtown_Toronto_grouped[Downtown_Toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

---- Adelaide----
                    venue  freq
0             Coffee Shop  0.07
1  Furniture / Home Store  0.06
2                     Spa  0.04
3        Asian Restaurant  0.04
4    Gym / Fitness Center  0.03


---- Bathurst Quay----
             venue  freq
0      Coffee Shop  0.19
1  Harbor / Marina  0.07
2             Café  0.07
3    Grocery Store  0.07
4             Park  0.07


---- Cabbagetown----
         venue  freq
0  Coffee Shop  0.10
1         Café  0.05
2  Pizza Place  0.05
3   Restaurant  0.05
4       Bakery  0.03


---- Chinatown----
                 venue  freq
0                  Bar  0.05
1                 Café  0.05
2  Arts & Crafts Store  0.04
3          Coffee Shop  0.04
4          Yoga Studio  0.04


---- Design Exchange----
         venue  freq
0  Coffee Shop  0.14
1        Hotel  0.06
2         Café  0.06
3   Restaurant  0.06
4          Gym  0.05


---- Grange Park----
                venue  freq
0                Café  0.09
1  Chinese Restaurant  0.09
2         C

In [124]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [125]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Downtown_Toronto_grouped['Neighborhood']

for ind in np.arange(Downtown_Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Downtown_Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adelaide,Coffee Shop,Furniture / Home Store,Spa,Asian Restaurant,Gym / Fitness Center,Sandwich Place,Boutique,Restaurant,Café,Yoga Studio
1,Bathurst Quay,Coffee Shop,Harbor / Marina,Café,Grocery Store,Park,Tunnel,Caribbean Restaurant,Sushi Restaurant,Sculpture Garden,Gym
2,Cabbagetown,Coffee Shop,Café,Pizza Place,Restaurant,Bakery,Diner,Pub,Gastropub,Pharmacy,Grocery Store
3,Chinatown,Bar,Café,Arts & Crafts Store,Coffee Shop,Yoga Studio,Park,Clothing Store,Vegetarian / Vegan Restaurant,Mexican Restaurant,Shoe Store
4,Design Exchange,Coffee Shop,Hotel,Café,Restaurant,Gym,Seafood Restaurant,American Restaurant,Deli / Bodega,Asian Restaurant,Thai Restaurant


In [126]:
# set number of clusters
kclusters = 5

Downtown_Toronto_grouped_clustering = Downtown_Toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Downtown_Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 4, 3, 0, 3, 3, 0, 3, 3, 4], dtype=int32)

In [127]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Downtown_Toronto_merged = Downtown_Toronto_data

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
Downtown_Toronto_merged = Downtown_Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Downtown_Toronto_merged.head() # check the last columns!

,index,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,2,M5A,Downtown Toronto,Regent Park,43.660690,-79.360310,3,Coffee Shop,Restaurant,Thai Restaurant,Pub,Auto Dealership,Intersection,Grocery Store,Food Truck,Fast Food Restaurant,Electronics Store
1,2,M5A,Downtown Toronto,Harbourfront,43.639230,-79.383070,3,Coffee Shop,Aquarium,Pizza Place,Hotel,Café,Brewery,Italian Restaurant,Restaurant,Scenic Lookout,Sporting Goods Shop
2,9,M5B,Downtown Toronto,Garden District,43.657940,-79.375620,3,Coffee Shop,Hotel,Middle Eastern Restaurant,Café,Sandwich Place,Diner,Bar,Lounge,Restaurant,Hookah Bar
3,9,M5B,Downtown Toronto,Ryerson,43.648847,-79.402492,0,Bar,Furniture / Home Store,Coffee Shop,Café,Yoga Studio,Boutique,Caribbean Restaurant,Pizza Place,Park,Spa
4,15,M5C,Downtown Toronto,St. James Town,43.671100,-79.373590,4,Coffee Shop,Grocery Store,Hotel,Pie Shop,Bistro,Breakfast Spot,Caribbean Restaurant,Sandwich Place,Pizza Place,Diner


In [115]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Downtown_Toronto_merged['Latitude'], Downtown_Toronto_merged['Longitude'], Downtown_Toronto_merged['Neighborhood'], Downtown_Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [110]:
Downtown_Toronto_merged.loc[Downtown_Toronto_merged['Cluster Labels'] == 0, Downtown_Toronto_merged.columns[[1] + list(range(5, Downtown_Toronto_merged.shape[1]))]]





,PostalCode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,M5B,-79.402492,0,Bar,Furniture / Home Store,Coffee Shop,Café,Yoga Studio,Boutique,Caribbean Restaurant,Pizza Place,Park,Spa
6,M5G,-79.390104,0,Italian Restaurant,Coffee Shop,Boutique,Restaurant,French Restaurant,Hotel,Spa,Café,Japanese Restaurant,Indian Restaurant
7,M6G,-79.422094,0,Grocery Store,Park,Gym,Nightclub,Baby Store,Coffee Shop,Candy Store,Beer Store,Café,American Restaurant
9,M5H,-79.401620,0,Coffee Shop,Furniture / Home Store,Spa,Asian Restaurant,Gym / Fitness Center,Sandwich Place,Boutique,Restaurant,Café,Yoga Studio
19,M5S,-79.417019,0,Korean Restaurant,Coffee Shop,Dessert Shop,Japanese Restaurant,Karaoke Bar,Grocery Store,Ice Cream Shop,Bubble Tea Shop,Café,Sandwich Place
20,M5T,-79.401610,0,Café,Coffee Shop,Bar,Vegetarian / Vegan Restaurant,Mexican Restaurant,Vietnamese Restaurant,Breakfast Spot,Bakery,Park,Ramen Restaurant
21,M5T,-79.397800,0,Bar,Café,Arts & Crafts Store,Coffee Shop,Yoga Studio,Park,Clothing Store,Vegetarian / Vegan Restaurant,Mexican Restaurant,Shoe Store
24,M5V,-79.395140,0,Restaurant,Coffee Shop,Italian Restaurant,Hotel,Beer Bar,Bar,Spa,Sandwich Place,Pizza Place,Park
28,M5V,-79.403743,0,Coffee Shop,Italian Restaurant,Bakery,Bar,Park,Restaurant,Brewery,Diner,Electronics Store,Gym / Fitness Center
30,M4W,-79.388980,0,Italian Restaurant,Yoga Studio,Butcher,Athletics & Sports,Bistro,Coffee Shop,Mattress Store,Salon / Barbershop,Café,Bar


In [111]:
Downtown_Toronto_merged.loc[Downtown_Toronto_merged['Cluster Labels'] == 1, Downtown_Toronto_merged.columns[[1] + list(range(5, Downtown_Toronto_merged.shape[1]))]]





,PostalCode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,M5H,-79.514337,1,Convenience Store,Moving Target,Park,Jewelry Store,Yoga Studio,Nail Salon,Noodle House,Nightclub,New American Restaurant,Music Store


In [112]:
Downtown_Toronto_merged.loc[Downtown_Toronto_merged['Cluster Labels'] == 2, Downtown_Toronto_merged.columns[[1] + list(range(5, Downtown_Toronto_merged.shape[1]))]]






,PostalCode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,M5J,-79.37661,2,Theme Park,Park,Pizza Place,Boat or Ferry,Burger Joint,Farm,Bar,Harbor / Marina,BBQ Joint,American Restaurant


In [113]:
Downtown_Toronto_merged.loc[Downtown_Toronto_merged['Cluster Labels'] == 3, Downtown_Toronto_merged.columns[[1] + list(range(5, Downtown_Toronto_merged.shape[1]))]]






,PostalCode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,-79.360310,3,Coffee Shop,Restaurant,Thai Restaurant,Pub,Auto Dealership,Intersection,Grocery Store,Food Truck,Fast Food Restaurant,Electronics Store
1,M5A,-79.383070,3,Coffee Shop,Aquarium,Pizza Place,Hotel,Café,Brewery,Italian Restaurant,Restaurant,Scenic Lookout,Sporting Goods Shop
2,M5B,-79.375620,3,Coffee Shop,Hotel,Middle Eastern Restaurant,Café,Sandwich Place,Diner,Bar,Lounge,Restaurant,Hookah Bar
5,M5E,-79.375170,3,Coffee Shop,Café,Hotel,Seafood Restaurant,Italian Restaurant,Bakery,Restaurant,Japanese Restaurant,Gym,Beer Bar
8,M5H,-79.383031,3,Coffee Shop,Café,Hotel,Clothing Store,Restaurant,Gym,Gastropub,Pizza Place,Salad Place,Japanese Restaurant
11,M5J,-79.383070,3,Coffee Shop,Aquarium,Pizza Place,Hotel,Café,Brewery,Italian Restaurant,Restaurant,Scenic Lookout,Sporting Goods Shop
12,M5J,-79.380630,3,Coffee Shop,Hotel,Sandwich Place,Café,Deli / Bodega,Japanese Restaurant,Breakfast Spot,Chinese Restaurant,Sushi Restaurant,Concert Hall
14,M5K,-79.381460,3,Coffee Shop,Hotel,Café,Japanese Restaurant,Italian Restaurant,Seafood Restaurant,Salad Place,Restaurant,Asian Restaurant,Sporting Goods Shop
15,M5K,-79.380440,3,Coffee Shop,Hotel,Café,Restaurant,Gym,Seafood Restaurant,American Restaurant,Deli / Bodega,Asian Restaurant,Thai Restaurant
16,M5L,-79.379515,3,Coffee Shop,Hotel,Café,Restaurant,Italian Restaurant,Gym,Japanese Restaurant,Seafood Restaurant,American Restaurant,Deli / Bodega


In [116]:
Downtown_Toronto_merged.loc[Downtown_Toronto_merged['Cluster Labels'] == 4, Downtown_Toronto_merged.columns[[1] + list(range(5, Downtown_Toronto_merged.shape[1]))]]






,PostalCode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,M5C,-79.373590,4,Coffee Shop,Grocery Store,Hotel,Pie Shop,Bistro,Breakfast Spot,Caribbean Restaurant,Sandwich Place,Pizza Place,Diner
27,M5V,-79.398110,4,Coffee Shop,Harbor / Marina,Café,Grocery Store,Park,Tunnel,Caribbean Restaurant,Sushi Restaurant,Sculpture Garden,Gym
29,M5V,-79.395851,4,Airport Lounge,Airport Service,Airport Terminal,Coffee Shop,Harbor / Marina,Sculpture Garden,Park,Airport,Airport Food Court,Grocery Store
31,M4X,-79.373590,4,Coffee Shop,Grocery Store,Hotel,Pie Shop,Bistro,Breakfast Spot,Caribbean Restaurant,Sandwich Place,Pizza Place,Diner
